In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from bisect import bisect_left
import warnings
warnings.filterwarnings("ignore")


def Kc_from_API(api, t):
  t = (9*t)/5 + 32
  c1 = 0.766506 + (0.0132417*t) - (1.02865*(t/100)*(t/100)) + (0.36681*math.pow(t/100, 3)) - ((5.9763*math.pow(t/100, 4))/100) + (3.7066*math.pow(t/100, 5) / 1000)
  c2 = -0.124687 + ((5.02722*t)/10000) - (2.69387*(t/1000)*(t/1000)) + (9.5991*math.pow(t/1000, 3)) - ((1.6992*math.pow(t/100, 4))/1000) + ((1.1212*math.pow(t/100, 5))/10000)
  c3_104 = 8.23768 - (0.0795248*t) + (4.67827*(t/100)*(t/100)) - (1.44508*math.pow(t/100, 3)) + (0.21488*math.pow(t/100, 4)) - (0.012001 * math.pow(t/100, 5))
  return math.exp(c1 + c2*api + c3_104*(api/100)*(api/100))

def find_prop(temp,prop,T,d=3,isval_temp=False):
  if isval_temp:
    for i in range(len(temp)):
      temp[i]= (temp[i] - 32)*5/9
  prop_relation=np.poly1d(np.polyfit(temp,prop, deg=d))
  return (prop_relation[3]*(T**3) + prop_relation[2]*(T**2) + prop_relation[1]*T + prop_relation[0])

def Cp_water(T):
  return (Cp_T[3]*(T**3) + Cp_T[2]*(T**2) + Cp_T[1]*T + Cp_T[0])*1000

def lmtd(T_hi,T_co,T_ho,T_ci):
  del_t1 = T_hi - T_co
  del_t2 = T_ho - T_ci
  return (del_t1 - del_t2)/(math.log(del_t1/del_t2))

def lmtd_correction(T_hi,T_co,T_ho,T_ci):
  Rn = T_hi - T_ho
  Rd = T_co - T_ci
  Sn = T_co - T_ci
  Sd = T_hi - T_ci
  rv = math.sqrt(Rn*Rn + Rd*Rd) #rough variable
  rv3 = rv
  rv = rv/(Rn-Rd)
  Ft1_2 = rv
  rv = Rd*(Sd-Sn)/(Rd*Sd - Rn*Sn)
  rv = math.log(rv)
  Ft1_2 *= rv
  Ft2_4 = Ft1_2 / 2
  rv = Sn * (Rn + Rd - rv3)
  rv = 2*Sd*Rd - rv
  rv2 = Sn * (Rn + Rd + rv3)
  rv2 = 2*Sd*Rd - rv2
  Ft1_2 /= math.log(rv/rv2)
  rv = 2*Rd*Sd - Sn*Rd - Rn*Sn
  rv += 2*math.sqrt(Rd*(Sd-Sn)*(Rd*Sd-Rn*Sn))
  Ft2_4 /= math.log((rv+Sn*rv3)/(rv-Sn*rv3))
  return Ft2_4,Ft1_2
 
def sq_tube_passes(nt, Nt):
  shell_id_in = [8.0, 10.0, 12.0, 13.25, 15.25, 17.25, 19.25, 21.25, 23.25, 25.0, 27.0, 29.0, 31.0, 33.0, 35.0, 37.0, 39.0]
  P2 = [16, 32, 45, 56, 76, 112, 132, 166, 208, 252, 288, 326, 398, 460, 518, 574, 644]
  P4 = [14, 26, 40, 52, 68, 96, 128, 158, 192, 238, 278, 300, 380, 432, 488, 562, 624]
  if Nt==1:
    i = bisect_left(P1, int(nt))
    return P1[i], shell_id_in[i]*0.0254
  elif Nt==2:
    i = bisect_left(P2, int(nt))
    return P2[i], shell_id_in[i]*0.0254

def jH_from_re(Re):
  return 0.806455 + 0.36921*pow(Re, 0.541044)

def TEMA_friction_factor(Re):
  if Re<2300:
    print("The final Re of Tube is",Re);
    print("The Tube side flow is Laminar.")
  if Re>25000:
    return (1/pow((0.0784538*math.log(Re+1.17926)+1.03963),5.58109)) - 0.00286896
  else:
    return 0.00269998+0.526772/pow(Re,0.288988)

#https://inside.mines.edu/~jjechura/Refining/02_Feedstocks_&_Products.pdf (API of heavy naptha)
print("\033[1mDesign parameters of a Shell & Tube Heat Exchanger \033[0m")
print()
print("Hot Fluid: Heavy Naphtha \nCold Fluid: Water")
val = input("Enter the mass flow rate of Heavy Naptha in kg/s(if you want to enter in 1.5+0.03*G format enter 'G'): ")
if val.lower() == 'g':
  m_naptha = 1.5+0.03*(int(input("Group Number: ")))
else:
  m_naptha = float(val)
print("-"*50)
print()
print("Mass Flow Rate of Heavy Naptha:", m_naptha,"kg/s")
print("Please input the following values in degree Celsius.")
T_hi = float(input("Initial Temperature of the Hot Fluid: "))
T_ho = float(input("Desired Final Temperature of the Hot Fluid: "))
T_ci = float(input("Coolant initial temperature: "))
T_co = float(input("Coolant final temperature: "))
print()
print('-'*50)
print()
del_t1 = T_hi-T_ho
del_t2= T_ho-T_hi
api_h = 47.3  
api_c = 10    

if del_t1<10:
  Fc = 1/2
else:
  Kc = Kc_from_API(api_h, del_t1)
  rn = T_ho - T_ci
  rd = T_hi - T_co
  Fc = Kc*rn/(rn-rd)
  rn = (math.log(Kc+1)/math.log(rn/rd))
  Fc = (Fc-rn)/(Kc+Kc*rn)
print("Caloric Fraction of Hot Fluid:","%.4f"%Fc)
T_hc = T_ho + Fc*(T_hi-T_ho)    #Caloric Temperature of Hot Fluid
print("Caloric Temperature Of Hot Fluid:","%.4f"%T_hc,"C")

del_t2= T_co-T_ci
if del_t2<10:
  Fc = 1/2
else:
  Kc = Kc_from_API(api_c, del_t2)
  rn = T_ho - T_ci
  rd = T_hi - T_co
  Fc = Kc*rn/(rn-rd)
  rn = (math.log(Kc+1)/math.log(rn/rd))
  Fc = (Fc-rn)/(Kc+Kc*rn)
print("Caloric Fraction of Cold Fluid:","%.4f"%Fc)
T_cc = T_ci + Fc*(T_co-T_ci) #Caloric Temperature Of Cold Fluid
print("Caloric Temperature Of Cold Fluid:","%.4f"%T_cc,"C")
print()
print("-"*50)
print()
print("\033[1mProperties of the Fluids at Tc  \033[0m")
print()
#https://m.cameochemicals.noaa.gov/chris/PTN.pdf

visc_naptha_temps=[104,122,212] #Units: F
visc_naptha=[1.04,0.92,0.58] #Units: cP

temp_naptha=[52,54,56,58,60,62,64,66,68,70,72,74,76,78,80,82,84,86] #Units : C
density_naptha=[46.750,46.680,46.610,46.540,46.470,46.400,46.330,46.260,46.190,46.120,46.050,45.980,45.920,45.850,45.780,45.710,45.640,45.570] #Units: lb/ft3

for i in range(len(density_naptha)):
  density_naptha[i]=16.0185*density_naptha[i] #Units: kg/m3


temp_water = list(range(20,41))
density_water = [0.9982,0.998,0.9978,0.9975,0.9973,0.997,0.9968,0.9965,0.9962,0.9959,0.9956,0.9953,0.995,0.9947,0.9944,0.994,0.9937,0.9933,0.993,0.9926,0.9922]
visc_water = [1.0016,0.9775,0.9544,0.9321,0.9107,0.89,0.8701,0.8509,0.8324,0.8145,0.7972,0.7805,0.7644,0.7488,0.7337,0.7191,0.705,0.6913,0.678,0.6652,0.6527]

visc_c= find_prop(temp_water,visc_water,T_cc)
density_c= find_prop(temp_water,density_water,T_cc)
visc_h= find_prop(visc_naptha_temps,visc_naptha,T_hc,3,True)
density_h= 141.5/(api_h+131.5)
k_h = 1.040*0.14422788886428 #Thermal Conductivity of Naphtha in SI, practically constant wrt T
k_c = 0.598 #Units:W/m·K
#Cp of Naptha
T_naptha = [ 10,  15,  20,  25,  30,  35,  40,  45,  50,  55,  60,  65,  70, 75,  80,  85,  90,  95, 100, 105]
naph_Cp = [0.456, 0.459, 0.461, 0.464, 0.467, 0.469, 0.472, 0.474, 0.477, 0.48 , 0.482, 0.485, 0.488, 0.49 , 0.493, 0.495, 0.498, 0.501, 0.503, 0.506]
Cp_T_naptha =np.poly1d(np.polyfit(T_naptha ,naph_Cp, deg=3))
def naptha_Cp(T,Cp_T_naptha):
  return (Cp_T_naptha[3]*(T**3) + Cp_T_naptha[2]*(T**2) + Cp_T_naptha[1]*T + Cp_T_naptha[0])*4186.8 

Cp_h=naptha_Cp(T_hc,Cp_T_naptha)
#Cp of water
#http://www.vaxasoftware.com/doc_eduen/qui/caloresph2o.pdf
x=range(20,41)
y=[4.182,4.181,4.181,4.180,4.180,4.180,4.179,4.179,4.179,4.179,4.178,4.178,4.178,4.178,4.178,4.178,4.178,4.178,4.178,4.179,4.179]
Cp_T=np.poly1d(np.polyfit(x,y, deg=3))
Cp_c=Cp_water(T_cc)#Units: J/kg⋅K
print("Hot Fluid: Heavy Naptha")
print("Specific heat Capacity of Hot Fluid:","%.4f"%Cp_h,"J/kg·K")
print("Viscosity of Hot Fluid:",visc_h,"cP") #Units: cP
print("Denisty of Hot Fluid:","%.4f"%(density_h*1000),"kg/m\u00b3") #Units: kg/m3
print("Thermal Conductivity of Hot Fluid:","%.4f"%k_h,"W/m·K")
print()
print("Cold Fluid: Water")
print("Specific Heat Capacity of Water:","%.4f"%Cp_c,"J/kg·K")
print("Viscosity of Cold Fluid:","%.4f"%visc_c,"cP") #Units: cP
print("Denisty of Cold Fluid:","%.4f"%(density_c*1000),"kg/m\u00b3") #Units: kg/m3
print("Thermal Conductivity of Cold Fluid:","%.4f"%k_c,"W/m·K")
print()
print("-"*50)
print()
print("\033[1mEnthalpy Heat Balance \033[0m")
print()
print("Heat Rate: ","%.4f"%(m_naptha*Cp_h*(T_hi-T_ho)),"J/kg·s")
m_water=(m_naptha*Cp_h*(T_hi-T_ho))/(Cp_c*(T_co-T_ci))
print("Mass Flow Rate of Water:","%.4f"%m_water,"kg/s")
print()
print("-"*50)
print()
print("\033[1mLMTD and LMTD Correction Factor \033[0m")
print()
lmtd=lmtd(T_hi,T_co,T_ho,T_ci)
print("LMTD considering counter-current flow:","%.4f"%lmtd,"C")
Ft2_4,Ft1_2=lmtd_correction(T_hi,T_co,T_ho,T_ci)
R = (T_hi-T_ho)/(T_co-T_ci)
S = (T_co-T_ci)/(T_hi-T_ci)
print("R:",R,"S:",S)
print("LMTD correction Factor for 1 shell pass and 2 or more tube passes")
print("Ft = ",'%.4f'%Ft1_2,sep='')
print("LMTD correction Factor for 2 shell pass and 4 or more tube passes")
print("Ft = ",'%.4f'%Ft2_4,sep='')

shell_pass,tube_pass=0,0
if Ft1_2<0.9:
      Ft = Ft2_4
      shell_pass = 2
      tube_pass = 4
      print("2-4 Heat Exchanger is the better option. The code will proceed calculating parameters for 2-4 HE.")
else:
      Ft = Ft1_2
      shell_pass = 1
      tube_pass = 2
      print("1-2 Heat Exchanger is the better option. As Ft>=0.9. The code will proceed calculating parameters for 1-2 HE.")
print()
print("-"*50)
print()
print("\033[1mShell Side and Tube Side Parameters \033[0m")
print()
U_assum = (50+125)*5.678263341123/2     #SI Units, W/((sq.m).K)
print("\033[4mIteration 0 \033[0m \nU(assumed):", "%.4f"%U_assum, "W/m\u00b2·K")
Area = (m_naptha*Cp_h*(T_hi-T_ho))/(U_assum*lmtd*Ft)
print("Area:", "%.4f"%Area, "m\u00b2")

do = 0.0254
di = 0.017018
visc_h,visc_c = visc_h*0.001,visc_c*0.001
tube_len = 20*0.3048
nt = Area/(math.pi*do*tube_len)
print("Required number of Tubes:", math.ceil(nt))

Pt = 1.25*0.0254               
Nt = 2 
na_sq, Ds_sq = sq_tube_passes(nt, Nt)
print("Standard Number of tubes:",na_sq)
print("Ds_sq:"'%.4f'%Ds_sq,"m")
print()
#Tube Side Calculations
print("Tube Side Fluid: Heavy Naptha")
Re_tube = (4*m_naptha*Nt)/(math.pi*di*visc_h*na_sq)
k_h = 1.040*0.14422788886428 #Thermal Conductivity of Naphtha in SI, practically constant wrt T
Pr = (Cp_h*visc_h/(k_h))
Nu = 0.023*(Re_tube**0.8)*(Pr**0.3)
hi = (k_h*Nu/di)
print("Tube Side Reynolds Number (Re|tube): "'%.4f'%Re_tube)
print("Tube Side Heat Transfer Coefficient (ho):"'%.4f'%hi,"W/m\u00b2·K")
print()
#Shell Side Calculations
print("Shell Side Fluid: Water")
B = Ds_sq/2    #Baffle spacing
C = Pt - do    #Tube Clearance
a_shell = C*B*Ds_sq/Pt    #Shell side cross flow area
D_eq = (4*Pt*Pt - math.pi*do*do)/(math.pi*do)
Re_shell = m_water*D_eq/a_shell/visc_c
print("Shell Side Reynolds Number (Re|shell): "'%.4f'%Re_shell)

k_c = 0.598 #Units:W/m·K
jH = jH_from_re(Re_shell)
Pr_shell = Cp_c*visc_c/k_c
ho = jH*k_c*(Pr_shell**(-1/3))/D_eq            #Conductive Heat Transfer of Naphtha
print("Shell Side Heat Transfer Coefficient (ho): ", '%.4f'%ho, "W/m\u00b2·K")

K_wall = 54           #Thermal Conductivity of Carbon Steel
Rdi =  0.00018         # Tube inside dirt fouling factor
Rdo = 0.00075        # Tube outside dirt fouling factor
Ucal = 9*hi*ho*K_wall
rough = (do-di)*hi + 2*Rdi*K_wall*hi + 2*K_wall
rough = 8 * rough * ho
rough = rough + 9*hi*K_wall*(1+Rdo*ho)
Ucal = Ucal/rough
print()
rel_error = abs((Ucal-U_assum)/U_assum)*100 # Tolerance of U
print("Ucal:", '%.4f'%Ucal, "W/m\u00b2·K", "\nRelative error = ", '%.4f'%rel_error, "%")
print("Since the relative error is >5% we proceed to do further iterations")

count=1
print()
while rel_error>=5:
  U_assum = Ucal
  Area = (m_naptha*Cp_h*(T_hi-T_ho))/(U_assum*lmtd*Ft)
  print("\033[4mIteration ",count,"\033[0m")
  nt = Area/(math.pi*do*tube_len)
  print("Required number of Tubes:", math.ceil(nt))
  na_sq, Ds_sq = sq_tube_passes(nt, Nt)
  print("Standard Number of tubes:",na_sq)
  print("Ds_sq:"'%.4f'%Ds_sq,"m")
  print()

  #Tube Side Calculations
  Re_tube = (4*m_naptha*Nt)/(math.pi*di*visc_h*na_sq)
  k_h = 1.040*0.14422788886428 #Thermal Conductivity of Naphtha in SI, practically constant wrt T
  Pr = (Cp_h*visc_h/(k_h))
  Nu = 0.023*(Re_tube**0.8)*(Pr**0.3)
  hi = (k_h*Nu/di)
  print("Tube Side Reynolds Number (Re|tube): "'%.4f'%Re_tube)
  print("Tube Side Heat Transfer Coefficient (hi):"'%.4f'%hi,"W/m\u00b2·K")

  #Shell Side Calculations
  B = Ds_sq/2    #Baffle spacing
  C = Pt - do    #Tube Clearance
  a_shell = C*B*Ds_sq/Pt    #Shell side cross flow area
  D_eq = (4*Pt*Pt - math.pi*do*do)/(math.pi*do)
  Re_shell = m_water*D_eq/a_shell/visc_c
  print("Shell Side Reynolds Number (Re|shell): "'%.4f'%Re_shell)

  jH = jH_from_re(Re_shell)
  Pr_shell = Cp_h*visc_c/k_c
  ho = jH*k_c*(Pr_shell**(-1/3))/D_eq            #Conductive Heat Transfer of Naphtha
  print("Shell Side Heat Transfer Coefficient (ho): ", '%.4f'%ho, "W/m\u00b2·K")

  Ucal = 9*hi*ho*K_wall
  rough = (do-di)*hi + 2*Rdi*K_wall*hi + 2*K_wall
  rough = 8 * rough * ho
  rough = rough + 9*hi*K_wall*(1+Rdo*ho)
  Ucal = Ucal/rough

  rel_error = abs((Ucal-U_assum)/U_assum)*100 # Tolerance of U
  print("Ucal:", '%.4f'%Ucal, "W/m\u00b2·K", "\nRelative error = ", '%.4f'%rel_error, "%")
  print()
  count+=1
print("-"*50)
print()
#Pressure Drop4
print("\033[1mPressure Drop and Overdesign\033[0m")
print()
f = TEMA_friction_factor(Re_tube)              #friction factor
g = 9.80665                                    #acceleration due to gravity
v_tube = 4 * m_naptha * Nt / (density_h*1000*math.pi*di*di*na_sq)       #tube side velocity
delta_Pf = 8 * m_naptha * m_naptha * f * Nt * Nt * tube_len / (math.pi*math.pi*pow(di,5)*na_sq*na_sq*density_h*1000)    #Frictional Pressure Drop (Darcy - Weisbatch Eq)
delta_Pr = 2 * Nt * v_tube * v_tube*1000 / (density_h*1000* g)
delta_P= delta_Pf + delta_Pr # calculation of pressure Drop
delta_P_psi = 0.0001450377 * delta_P
print("Tube side velocity: "'%.4f'%v_tube,"m/s")
print("Friction Factor: "'%.5f'%f)
print("delta Pf:"'%.4f'%delta_Pf,"Pa", ", delta_Pr: "'%.6f'%delta_Pr,"Pa")
print("Tube Side Pressure Drop: " '%.4f'%delta_P_psi,"psi","(", '%.4f'%delta_P, "Pa )")
print("Required Number of Tubes:",math.ceil(nt))
print("Standard Number of Tubes:",na_sq)
overdesign = abs((na_sq-math.ceil(nt))/math.ceil(nt))*100   # calculation of area overdesign
print("%Overdesign:",'%.4f'%overdesign,"%")
print()
if(overdesign<10):
  print("\033[1mDesign is acceptable and cost-effective\033[0m")
else:
  print("Consider different dimesnions of tubes, Uassumed inital could be varied for acceptable design")